## Import Library

In [1]:
import csv
import re
import nltk
import Sastrawi
import sklearn
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Dataset

In [2]:
df = pd.read_csv('data/tweets_labeled.csv')
df

,,tweet,target
0,2022-01-27,#AyoTolakUUIKN \n\nProyek IKN membuka peluang ...,0.0
1,2022-01-27,Tolak UU IKN karena berpotensi merusak lingkun...,0.0
2,2022-01-27,UU IKN hanya akan merugikan rakyat dengan huta...,0.0
3,2022-01-27,Jika UU ini diterapkan yang terjadi adalah mas...,0.0
4,2022-01-27,UU IKN Syarat kepentingan oligarki. Tolak n ba...,0.0
...,...,...,...
5888,2021-10-01,Horee!! Kabar gembira gaes!\nProses pemindahan...,1.0
5889,2021-10-01,Juru Bicara Presiden RI Fadjroel Rachman menga...,1.0
5890,2021-10-01,Pemerintah Indonesia mengajak Korea Selatan un...,1.0
5891,2021-10-01,@pikiran_rakyat masalah Jakarta tepatnya harus...,0.0


In [3]:
df.isnull().sum()

          0
tweet     0
target    1
dtype: int64

In [4]:
df = df.dropna()
df = df.reset_index(drop=True)

## Preprocessing

### Cleansing Data

In [5]:
import string 
import re #regex library

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention @
    text = re.sub(r"[@][\w_-]+","", text)
    # remove link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['cleansing'] = df['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['cleansing'] = df['cleansing'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['cleansing'] = df['cleansing'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['cleansing'] = df['cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['cleansing'] = df['cleansing'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['cleansing'] = df['cleansing'].apply(remove_singl_char)

# remove duplicate
#df.drop_duplicates(subset=['cleansing'], inplace=True)

df.loc[0:100,['tweet', 'cleansing']]

,tweet,cleansing
0,#AyoTolakUUIKN \n\nProyek IKN membuka peluang ...,Proyek IKN membuka peluang besar para oligarki...
1,Tolak UU IKN karena berpotensi merusak lingkun...,Tolak UU IKN karena berpotensi merusak lingkun...
2,UU IKN hanya akan merugikan rakyat dengan huta...,UU IKN hanya akan merugikan rakyat dengan huta...
3,Jika UU ini diterapkan yang terjadi adalah mas...,Jika UU ini diterapkan yang terjadi adalah mas...
4,UU IKN Syarat kepentingan oligarki. Tolak n ba...,UU IKN Syarat kepentingan oligarki Tolak bata...
...,...,...
96,Jangan sampai ibu kota baru jadi ibu tiri yang...,Jangan sampai ibu kota baru jadi ibu tiri yang...
97,"Untuk siapa IKN sebenarnya??? Dari analisa, ti...",Untuk siapa IKN sebenarnya Dari analisa tidak ...
98,UU IKN adalah bentuk legitimasi hukum atas ker...,UU IKN adalah bentuk legitimasi hukum atas ker...
99,Pemindahan Ibu Kota ditengah karut marut ekono...,Pemindahan Ibu Kota ditengah karut marut ekono...


In [6]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
display(df.loc[0:100,['tweet', 'cleansing']])

,tweet,cleansing
0,"#AyoTolakUUIKN \n\nProyek IKN membuka peluang besar para oligarki untuk bermain curang, securang²nya. Tolak UU IKN",Proyek IKN membuka peluang besar para oligarki untuk bermain curang securangnya Tolak UU IKN
1,Tolak UU IKN karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat. &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; IKN Proyek Oligarki &gt;&gt;,Tolak UU IKN karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat gtgt ltlt IKN Proyek Oligarki gtgt
2,UU IKN hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk. &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; IKN Proyek Oligarki &gt;&gt;,UU IKN hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk gtgt ltlt IKN Proyek Oligarki gtgt
3,Jika UU ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan. &gt;&gt; #AyoTolakUUIKN &lt;&lt; Tolak UU IKN &gt;&gt;,Jika UU ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan gtgt ltlt Tolak UU IKN gtgt
4,UU IKN Syarat kepentingan oligarki. Tolak n batalkan saja &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; #StopKebijakanDzalim #fxnw,UU IKN Syarat kepentingan oligarki Tolak batalkan saja gtgt ltlt
...,...,...
96,Jangan sampai ibu kota baru jadi ibu tiri yang kerjanya menyengsarakan rakyat dengan hutang APBN yang semakin menggunung. &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; IKN Proyek Oligarki #zmp6,Jangan sampai ibu kota baru jadi ibu tiri yang kerjanya menyengsarakan rakyat dengan hutang APBN yang semakin menggunung gtgt ltlt IKN Proyek Oligarki
97,"Untuk siapa IKN sebenarnya??? Dari analisa, tidak ada hal yang mendesak sehingga harus pindah ibukota segera &gt;&gt; #AyoTolakUUIKN &lt;&lt; #UUIKNMeresahkan #e3zc",Untuk siapa IKN sebenarnya Dari analisa tidak ada hal yang mendesak sehingga harus pindah ibukota segera gtgt ltlt
98,UU IKN adalah bentuk legitimasi hukum atas kerusakan yang disebabkan oleh kebijakan pemerintah. &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; #uidr,UU IKN adalah bentuk legitimasi hukum atas kerusakan yang disebabkan oleh kebijakan pemerintah gtgt ltlt
99,Pemindahan Ibu Kota ditengah karut marut ekonomi?? NO WAY!! &gt;&gt; #AyoTolakUUIKN &lt;&lt; Tolak UU IKN &gt;&gt;,Pemindahan Ibu Kota ditengah karut marut ekonomi NO WAY gtgt ltlt Tolak UU IKN gtgt


### Case Folding

In [7]:
# ------ Case Folding --------
df['case_folding'] = df['cleansing'].str.lower()
df.loc[0:100,['cleansing', 'case_folding']]

,cleansing,case_folding
0,Proyek IKN membuka peluang besar para oligarki untuk bermain curang securangnya Tolak UU IKN,proyek ikn membuka peluang besar para oligarki untuk bermain curang securangnya tolak uu ikn
1,Tolak UU IKN karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat gtgt ltlt IKN Proyek Oligarki gtgt,tolak uu ikn karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat gtgt ltlt ikn proyek oligarki gtgt
2,UU IKN hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk gtgt ltlt IKN Proyek Oligarki gtgt,uu ikn hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk gtgt ltlt ikn proyek oligarki gtgt
3,Jika UU ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan gtgt ltlt Tolak UU IKN gtgt,jika uu ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan gtgt ltlt tolak uu ikn gtgt
4,UU IKN Syarat kepentingan oligarki Tolak batalkan saja gtgt ltlt,uu ikn syarat kepentingan oligarki tolak batalkan saja gtgt ltlt
...,...,...
96,Jangan sampai ibu kota baru jadi ibu tiri yang kerjanya menyengsarakan rakyat dengan hutang APBN yang semakin menggunung gtgt ltlt IKN Proyek Oligarki,jangan sampai ibu kota baru jadi ibu tiri yang kerjanya menyengsarakan rakyat dengan hutang apbn yang semakin menggunung gtgt ltlt ikn proyek oligarki
97,Untuk siapa IKN sebenarnya Dari analisa tidak ada hal yang mendesak sehingga harus pindah ibukota segera gtgt ltlt,untuk siapa ikn sebenarnya dari analisa tidak ada hal yang mendesak sehingga harus pindah ibukota segera gtgt ltlt
98,UU IKN adalah bentuk legitimasi hukum atas kerusakan yang disebabkan oleh kebijakan pemerintah gtgt ltlt,uu ikn adalah bentuk legitimasi hukum atas kerusakan yang disebabkan oleh kebijakan pemerintah gtgt ltlt
99,Pemindahan Ibu Kota ditengah karut marut ekonomi NO WAY gtgt ltlt Tolak UU IKN gtgt,pemindahan ibu kota ditengah karut marut ekonomi no way gtgt ltlt tolak uu ikn gtgt


### Tokenize

In [8]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tokenize'] = df['case_folding'].apply(word_tokenize_wrapper)

df.loc[0:100,['case_folding', 'tokenize']]

,case_folding,tokenize
0,proyek ikn membuka peluang besar para oligarki untuk bermain curang securangnya tolak uu ikn,"[proyek, ikn, membuka, peluang, besar, para, oligarki, untuk, bermain, curang, securangnya, tolak, uu, ikn]"
1,tolak uu ikn karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat gtgt ltlt ikn proyek oligarki gtgt,"[tolak, uu, ikn, karena, berpotensi, merusak, lingkungan, dan, tidak, ada, manfaat, bagi, masyarakat, gtgt, ltlt, ikn, proyek, oligarki, gtgt]"
2,uu ikn hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk gtgt ltlt ikn proyek oligarki gtgt,"[uu, ikn, hanya, akan, merugikan, rakyat, dengan, hutang, riba, negara, yang, kian, menumpuk, gtgt, ltlt, ikn, proyek, oligarki, gtgt]"
3,jika uu ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan gtgt ltlt tolak uu ikn gtgt,"[jika, uu, ini, diterapkan, yang, terjadi, adalah, masyarakat, akan, semakin, merasakan, kerugian, dan, kesengsaraan, gtgt, ltlt, tolak, uu, ikn, gtgt]"
4,uu ikn syarat kepentingan oligarki tolak batalkan saja gtgt ltlt,"[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan, saja, gtgt, ltlt]"
...,...,...
96,jangan sampai ibu kota baru jadi ibu tiri yang kerjanya menyengsarakan rakyat dengan hutang apbn yang semakin menggunung gtgt ltlt ikn proyek oligarki,"[jangan, sampai, ibu, kota, baru, jadi, ibu, tiri, yang, kerjanya, menyengsarakan, rakyat, dengan, hutang, apbn, yang, semakin, menggunung, gtgt, ltlt, ikn, proyek, oligarki]"
97,untuk siapa ikn sebenarnya dari analisa tidak ada hal yang mendesak sehingga harus pindah ibukota segera gtgt ltlt,"[untuk, siapa, ikn, sebenarnya, dari, analisa, tidak, ada, hal, yang, mendesak, sehingga, harus, pindah, ibukota, segera, gtgt, ltlt]"
98,uu ikn adalah bentuk legitimasi hukum atas kerusakan yang disebabkan oleh kebijakan pemerintah gtgt ltlt,"[uu, ikn, adalah, bentuk, legitimasi, hukum, atas, kerusakan, yang, disebabkan, oleh, kebijakan, pemerintah, gtgt, ltlt]"
99,pemindahan ibu kota ditengah karut marut ekonomi no way gtgt ltlt tolak uu ikn gtgt,"[pemindahan, ibu, kota, ditengah, karut, marut, ekonomi, no, way, gtgt, ltlt, tolak, uu, ikn, gtgt]"


### Normalisasi

In [9]:
normalized_word = pd.read_csv("references/NormalisasiKata/kamus_alay.csv")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

df['normalisasi'] = df['tokenize'].apply(normalized_term)

df.loc[0:100,['tokenize', 'normalisasi']]

,tokenize,normalisasi
0,"[proyek, ikn, membuka, peluang, besar, para, oligarki, untuk, bermain, curang, securangnya, tolak, uu, ikn]","[proyek, ikn, membuka, peluang, besar, para, oligarki, untuk, bermain, curang, securangnya, tolak, uu, ikn]"
1,"[tolak, uu, ikn, karena, berpotensi, merusak, lingkungan, dan, tidak, ada, manfaat, bagi, masyarakat, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[tolak, uu, ikn, karena, berpotensi, merusak, lingkungan, dan, tidak, ada, manfaat, bagi, masyarakat, gtgt, ltlt, ikn, proyek, oligarki, gtgt]"
2,"[uu, ikn, hanya, akan, merugikan, rakyat, dengan, hutang, riba, negara, yang, kian, menumpuk, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[uu, ikn, hanya, akan, merugikan, rakyat, dengan, hutang, riba, negara, yang, kian, menumpuk, gtgt, ltlt, ikn, proyek, oligarki, gtgt]"
3,"[jika, uu, ini, diterapkan, yang, terjadi, adalah, masyarakat, akan, semakin, merasakan, kerugian, dan, kesengsaraan, gtgt, ltlt, tolak, uu, ikn, gtgt]","[jika, uu, ini, diterapkan, yang, terjadi, adalah, masyarakat, akan, semakin, merasakan, kerugian, dan, kesengsaraan, gtgt, ltlt, tolak, uu, ikn, gtgt]"
4,"[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan, saja, gtgt, ltlt]","[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan, saja, gtgt, ltlt]"
...,...,...
96,"[jangan, sampai, ibu, kota, baru, jadi, ibu, tiri, yang, kerjanya, menyengsarakan, rakyat, dengan, hutang, apbn, yang, semakin, menggunung, gtgt, ltlt, ikn, proyek, oligarki]","[jangan, sampai, ibu, kota, baru, jadi, ibu, tiri, yang, kerjanya, menyengsarakan, rakyat, dengan, hutang, apbn, yang, semakin, menggunung, gtgt, ltlt, ikn, proyek, oligarki]"
97,"[untuk, siapa, ikn, sebenarnya, dari, analisa, tidak, ada, hal, yang, mendesak, sehingga, harus, pindah, ibukota, segera, gtgt, ltlt]","[untuk, siapa, ikn, sebenarnya, dari, analisa, tidak, ada, hal, yang, mendesak, sehingga, harus, pindah, ibukota, segera, gtgt, ltlt]"
98,"[uu, ikn, adalah, bentuk, legitimasi, hukum, atas, kerusakan, yang, disebabkan, oleh, kebijakan, pemerintah, gtgt, ltlt]","[uu, ikn, adalah, bentuk, legitimasi, hukum, atas, kerusakan, yang, disebabkan, oleh, kebijakan, pemerintah, gtgt, ltlt]"
99,"[pemindahan, ibu, kota, ditengah, karut, marut, ekonomi, no, way, gtgt, ltlt, tolak, uu, ikn, gtgt]","[pemindahan, ibu, kota, ditengah, karut, marut, ekonomi, no, way, gtgt, ltlt, tolak, uu, ikn, gtgt]"


### Stopwords

In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'gtgt','ltlt'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    #kata = [word for word in words if word not in list_stopwords]
    return [word for word in words if word not in list_stopwords]

df['stopwords'] = df['normalisasi'].apply(stopwords_removal) 

df.loc[0:100,['normalisasi', 'stopwords']]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reikiko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,normalisasi,stopwords
0,"[proyek, ikn, membuka, peluang, besar, para, oligarki, untuk, bermain, curang, securangnya, tolak, uu, ikn]","[proyek, ikn, membuka, peluang, oligarki, bermain, curang, securangnya, tolak, uu, ikn]"
1,"[tolak, uu, ikn, karena, berpotensi, merusak, lingkungan, dan, tidak, ada, manfaat, bagi, masyarakat, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[tolak, uu, ikn, berpotensi, merusak, lingkungan, manfaat, masyarakat, ikn, proyek, oligarki]"
2,"[uu, ikn, hanya, akan, merugikan, rakyat, dengan, hutang, riba, negara, yang, kian, menumpuk, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[uu, ikn, merugikan, rakyat, hutang, riba, negara, kian, menumpuk, ikn, proyek, oligarki]"
3,"[jika, uu, ini, diterapkan, yang, terjadi, adalah, masyarakat, akan, semakin, merasakan, kerugian, dan, kesengsaraan, gtgt, ltlt, tolak, uu, ikn, gtgt]","[uu, diterapkan, masyarakat, merasakan, kerugian, kesengsaraan, tolak, uu, ikn]"
4,"[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan, saja, gtgt, ltlt]","[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan]"
...,...,...
96,"[jangan, sampai, ibu, kota, baru, jadi, ibu, tiri, yang, kerjanya, menyengsarakan, rakyat, dengan, hutang, apbn, yang, semakin, menggunung, gtgt, ltlt, ikn, proyek, oligarki]","[kota, tiri, kerjanya, menyengsarakan, rakyat, hutang, apbn, menggunung, ikn, proyek, oligarki]"
97,"[untuk, siapa, ikn, sebenarnya, dari, analisa, tidak, ada, hal, yang, mendesak, sehingga, harus, pindah, ibukota, segera, gtgt, ltlt]","[ikn, analisa, mendesak, pindah, ibukota]"
98,"[uu, ikn, adalah, bentuk, legitimasi, hukum, atas, kerusakan, yang, disebabkan, oleh, kebijakan, pemerintah, gtgt, ltlt]","[uu, ikn, bentuk, legitimasi, hukum, kerusakan, disebabkan, kebijakan, pemerintah]"
99,"[pemindahan, ibu, kota, ditengah, karut, marut, ekonomi, no, way, gtgt, ltlt, tolak, uu, ikn, gtgt]","[pemindahan, kota, ditengah, karut, marut, ekonomi, way, tolak, uu, ikn]"


### Stemming

In [11]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['stemming'] = df['stopwords'].swifter.apply(get_stemmed_term)

df.loc[0:100,['stopwords', 'stemming']]

Pandas Apply:   0%|          | 0/5892 [00:00<?, ?it/s]

,stopwords,stemming
0,"[proyek, ikn, membuka, peluang, oligarki, bermain, curang, securangnya, tolak, uu, ikn]","[proyek, ikn, buka, peluang, oligarki, main, curang, curang, tolak, uu, ikn]"
1,"[tolak, uu, ikn, berpotensi, merusak, lingkungan, manfaat, masyarakat, ikn, proyek, oligarki]","[tolak, uu, ikn, potensi, rusak, lingkung, manfaat, masyarakat, ikn, proyek, oligarki]"
2,"[uu, ikn, merugikan, rakyat, hutang, riba, negara, kian, menumpuk, ikn, proyek, oligarki]","[uu, ikn, rugi, rakyat, hutang, riba, negara, kian, tumpuk, ikn, proyek, oligarki]"
3,"[uu, diterapkan, masyarakat, merasakan, kerugian, kesengsaraan, tolak, uu, ikn]","[uu, terap, masyarakat, rasa, rugi, sengsara, tolak, uu, ikn]"
4,"[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan]","[uu, ikn, syarat, penting, oligarki, tolak, batal]"
...,...,...
96,"[kota, tiri, kerjanya, menyengsarakan, rakyat, hutang, apbn, menggunung, ikn, proyek, oligarki]","[kota, tiri, kerja, sengsara, rakyat, hutang, apbn, gunung, ikn, proyek, oligarki]"
97,"[ikn, analisa, mendesak, pindah, ibukota]","[ikn, analisa, desak, pindah, ibukota]"
98,"[uu, ikn, bentuk, legitimasi, hukum, kerusakan, disebabkan, kebijakan, pemerintah]","[uu, ikn, bentuk, legitimasi, hukum, rusa, sebab, bijak, perintah]"
99,"[pemindahan, kota, ditengah, karut, marut, ekonomi, way, tolak, uu, ikn]","[pindah, kota, tengah, karut, marut, ekonomi, way, tolak, uu, ikn]"


In [14]:
df['label'] = df['target']
df

,,tweet,target,cleansing,case_folding,tokenize,normalisasi,stopwords,stemming,label
0,2022-01-27,"#AyoTolakUUIKN \n\nProyek IKN membuka peluang besar para oligarki untuk bermain curang, securang²nya. Tolak UU IKN",0.0,Proyek IKN membuka peluang besar para oligarki untuk bermain curang securangnya Tolak UU IKN,proyek ikn membuka peluang besar para oligarki untuk bermain curang securangnya tolak uu ikn,"[proyek, ikn, membuka, peluang, besar, para, oligarki, untuk, bermain, curang, securangnya, tolak, uu, ikn]","[proyek, ikn, membuka, peluang, besar, para, oligarki, untuk, bermain, curang, securangnya, tolak, uu, ikn]","[proyek, ikn, membuka, peluang, oligarki, bermain, curang, securangnya, tolak, uu, ikn]","[proyek, ikn, buka, peluang, oligarki, main, curang, curang, tolak, uu, ikn]",0.0
1,2022-01-27,Tolak UU IKN karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat. &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; IKN Proyek Oligarki &gt;&gt;,0.0,Tolak UU IKN karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat gtgt ltlt IKN Proyek Oligarki gtgt,tolak uu ikn karena berpotensi merusak lingkungan dan tidak ada manfaat bagi masyarakat gtgt ltlt ikn proyek oligarki gtgt,"[tolak, uu, ikn, karena, berpotensi, merusak, lingkungan, dan, tidak, ada, manfaat, bagi, masyarakat, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[tolak, uu, ikn, karena, berpotensi, merusak, lingkungan, dan, tidak, ada, manfaat, bagi, masyarakat, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[tolak, uu, ikn, berpotensi, merusak, lingkungan, manfaat, masyarakat, ikn, proyek, oligarki]","[tolak, uu, ikn, potensi, rusak, lingkung, manfaat, masyarakat, ikn, proyek, oligarki]",0.0
2,2022-01-27,UU IKN hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk. &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; IKN Proyek Oligarki &gt;&gt;,0.0,UU IKN hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk gtgt ltlt IKN Proyek Oligarki gtgt,uu ikn hanya akan merugikan rakyat dengan hutang riba negara yang kian menumpuk gtgt ltlt ikn proyek oligarki gtgt,"[uu, ikn, hanya, akan, merugikan, rakyat, dengan, hutang, riba, negara, yang, kian, menumpuk, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[uu, ikn, hanya, akan, merugikan, rakyat, dengan, hutang, riba, negara, yang, kian, menumpuk, gtgt, ltlt, ikn, proyek, oligarki, gtgt]","[uu, ikn, merugikan, rakyat, hutang, riba, negara, kian, menumpuk, ikn, proyek, oligarki]","[uu, ikn, rugi, rakyat, hutang, riba, negara, kian, tumpuk, ikn, proyek, oligarki]",0.0
3,2022-01-27,Jika UU ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan. &gt;&gt; #AyoTolakUUIKN &lt;&lt; Tolak UU IKN &gt;&gt;,0.0,Jika UU ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan gtgt ltlt Tolak UU IKN gtgt,jika uu ini diterapkan yang terjadi adalah masyarakat akan semakin merasakan kerugian dan kesengsaraan gtgt ltlt tolak uu ikn gtgt,"[jika, uu, ini, diterapkan, yang, terjadi, adalah, masyarakat, akan, semakin, merasakan, kerugian, dan, kesengsaraan, gtgt, ltlt, tolak, uu, ikn, gtgt]","[jika, uu, ini, diterapkan, yang, terjadi, adalah, masyarakat, akan, semakin, merasakan, kerugian, dan, kesengsaraan, gtgt, ltlt, tolak, uu, ikn, gtgt]","[uu, diterapkan, masyarakat, merasakan, kerugian, kesengsaraan, tolak, uu, ikn]","[uu, terap, masyarakat, rasa, rugi, sengsara, tolak, uu, ikn]",0.0
4,2022-01-27,UU IKN Syarat kepentingan oligarki. Tolak n batalkan saja &gt;&gt; #UUIKNMendzalimiRakyat &lt;&lt; #StopKebijakanDzalim #fxnw,0.0,UU IKN Syarat kepentingan oligarki Tolak batalkan saja gtgt ltlt,uu ikn syarat kepentingan oligarki tolak batalkan saja gtgt ltlt,"[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan, saja, gtgt, ltlt]","[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan, saja, gtgt, ltlt]","[uu, ikn, syarat, kepentingan, oligarki, tolak, batalkan]","[uu, ikn, syarat, penting, oligarki, tolak, batal]",0.0
...,...

### Final Preprocessing

In [13]:
df_post_prepro = df[['tweet','stemming','label']].copy(deep=True)

KeyError: "['label'] not in index"

In [ ]:
df_post_prepro

### Export to CSV

In [ ]:
df_post_prepro.to_csv(r'/Users/reikiko/Documents/Study/TA/data/clean_tweets.csv',index=False)

In [ ]:
df_post_prepro.isnull().sum()